# Galaxy plots

This version matches galaxies with both subhalos and halos.  
However at this stage, there are no properties stored in either subhalos or halos apart from mass (I think).

In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='galaxy.yml'
displayParameters=True

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

haloFile=parameters['inputFiles']['haloFile']
subhaloFile=parameters['inputFiles']['subhaloFile']
galaxyFile=parameters['inputFiles']['galaxyFile']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']
baryonFraction=parameters['cosmo']['baryonFraction']

In [ ]:
# Read in halo table
with h5py.File(haloFile,'r') as f:
    #Get the data from the snapshot
    halos=f['Halos'][:]
print('halos.dtype =',halos.dtype)
    
# Read in subhalo table
with h5py.File(subhaloFile,'r') as f:
    #Get the data from the snapshot
    subhalos=f['Halos'][:]
print('subhalos.dtype =',subhalos.dtype)

# Read in galaxy table
with h5py.File(galaxyFile,'r') as f:    
    #Get the data from the snapshot
    galaxies=f['Galaxies'][:]
print('galaxies.dtype =',galaxies.dtype)

In [ ]:
# Filter out the halos, subhalos and galaxies in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
halos=halos[(graphMin<=halos['graph_ID']) & (halos['graph_ID']<=graphMax) & 
            (snapMin<=halos['snap_ID']) & (halos['snap_ID']<=snapMax)]
subhalos=subhalos[(graphMin<=subhalos['graph_ID']) & (subhalos['graph_ID']<=graphMax) & 
            (snapMin<=subhalos['snap_ID']) & (subhalos['snap_ID']<=snapMax)]
galaxies=galaxies[(graphMin<=galaxies['graph_ID']) & (galaxies['graph_ID']<=graphMax) & 
            (snapMin<=galaxies['snap_ID']) & (galaxies['snap_ID']<=snapMax)]

In [ ]:
# Match up galaxies with subhalos and/or halos
sub_ID_loc=np.where(np.logical_and(galaxies['sub_ID']==subhalos['sub_ID'],galaxies['halo_ID']==halos['halo_ID']))[0]


In [ ]:
print(halos['halo_ID'])
print(galaxies['halo_ID'])
print(subhalos['sub_ID'])
print(galaxies['sub_ID'])

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='baryonFraction':
            xdata=halos['mass']
            ydata=halos['mass_baryon']/halos['mass']
            plt.plot(xdata,0.*xdata+baryonFraction,'-')
        elif plotName=='stellarHaloMassRatio':
            xdata=halos['mass']
            ydata=halos['mass_stars']/halos['mass']
        elif plotName=='starFormationRate':
            xdata=halos['mass']
            ydata=halos['star_formation_rate']
        else:
            print('Plot type',plotName,'not yet implemented')
            continue # with next plot request
        plt.plot(xdata,ydata,plotProps['markerType'])

In [ ]:
baryonFraction